In [ ]:
from flask import Flask, request, jsonify, send_file, redirect, url_for, render_template_string
import os, csv, tempfile, threading, time, statistics, io
import matplotlib.pyplot as plt
from typing import List, Dict
from datetime import datetime

FILE_NAME = "students.txt"
BACKUP_DIR = "backups"
AUDIT_LOG = "audit.log"
REPORT_FILE = "students_report.txt"
ADMIN_TOKEN = "hackathon_secret_token_123"
LOCK = threading.Lock()

VALID_GRADES = set("ABCDEF")

INDEX_HTML = """
<!doctype html>
<title>Smart Student Management System</title>
<h1>Smart Student Management System (File-based)</h1>
<p>
  <a href="/ui/view">View Students</a> |
  <a href="/ui/add">Add Student</a> |
  <a href="/ui/analyze">Analyze</a> |
  <a href="/ui/seed">Seed Demo</a> |
  <a href="/ui/export">Export CSV</a>
</p>
<hr>
<div>{{ body|safe }}</div>
"""

VIEW_HTML = """
<h2>All Students</h2>
<table border="1" cellpadding="6">
<tr><th>ID</th><th>Name</th><th>Age</th><th>Grade</th><th>Marks</th><th>Actions</th></tr>
{% for s in students %}
<tr>
<td>{{s['id']}}</td>
<td>{{s['name']}}</td>
<td>{{s['age']}}</td>
<td>{{s['grade']}}</td>
<td>{{s['marks']}}</td>
<td>
  <a href="/ui/edit/{{s['id']}}">Edit</a> |
  <a href="/ui/delete/{{s['id']}}">Delete</a>
</td>
</tr>
{% endfor %}
</table>
"""

ADD_HTML = """
<h2>Add Student</h2>
<form method="post">
ID: <input name="id"><br>
Name: <input name="name"><br>
Age: <input name="age"><br>
Grade: <input name="grade"><br>
Marks: <input name="marks"><br>
<input type="submit" value="Add">
</form>
"""

EDIT_HTML = """
<h2>Edit Student</h2>
<form method="post">
ID: <b>{{s['id']}}</b><br>
Name: <input name="name" value="{{s['name']}}"><br>
Age: <input name="age" value="{{s['age']}}"><br>
Grade: <input name="grade" value="{{s['grade']}}"><br>
Marks: <input name="marks" value="{{s['marks']}}"><br>
<input type="submit" value="Update">
</form>
"""

ANALYZE_HTML = """
<h2>Analysis</h2>
<pre>{{ report }}</pre>
<p>
  <img src="/chart/marks.png" alt="marks histogram"><br>
  <img src="/chart/grades.png" alt="grade distribution">
</p>
"""

app = Flask(__name__)

def ensure_dirs():
    if not os.path.exists(BACKUP_DIR):
        os.makedirs(BACKUP_DIR)

def log_audit(action: str, detail: str):
    ts = datetime.utcnow().isoformat()
    line = f"{ts} | {action} | {detail}\n"
    with LOCK:
        with open(AUDIT_LOG, "a", encoding="utf-8") as f:
            f.write(line)

def read_students() -> List[Dict]:
    students = []
    if not os.path.exists(FILE_NAME):
        return students
    with LOCK:
        with open(FILE_NAME, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) != 5:
                    continue
                sid, name, age_s, grade_s, marks_s = [c.strip() for c in row]
                try:
                    age = int(age_s); marks = int(marks_s); grade = grade_s.upper()
                    if not sid.isdigit(): continue
                    if grade not in VALID_GRADES: continue
                except:
                    continue
                students.append({'id': sid, 'name': name, 'age': age, 'grade': grade, 'marks': marks})
    return students

def atomic_write(students: List[Dict]):
    ensure_dirs()
    fd, tmp = tempfile.mkstemp(prefix="students_", text=True)
    try:
        with os.fdopen(fd, "w", newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for s in students:
                writer.writerow([s['id'], s['name'], str(s['age']), s['grade'], str(s['marks'])])
        os.replace(tmp, FILE_NAME)
    finally:
        if os.path.exists(tmp):
            try: os.remove(tmp)
            except: pass

def backup_now():
    ensure_dirs()
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    target = os.path.join(BACKUP_DIR, f"students_{ts}.bak")
    with LOCK:
        if os.path.exists(FILE_NAME):
            with open(FILE_NAME, "rb") as src, open(target, "wb") as dst:
                dst.write(src.read())
    log_audit("backup", f"backup -> {target}")
    return target

def validate_record(rec: Dict, check_unique=True) -> (bool, str):
    if not rec.get('id') or not rec['id'].isdigit():
        return False, "ID must be integer"
    students = read_students()
    if check_unique and any(s['id']==rec['id'] for s in students):
        return False, "ID exists"
    if not rec.get('name') or rec['name'].strip()=="":
        return False, "Name empty"
    try:
        a = int(rec.get('age', -1))
        if a <= 0 or a>120: return False, "Invalid age"
    except:
        return False, "Invalid age"
    g = rec.get('grade','').upper()
    if len(g)!=1 or g not in VALID_GRADES:
        return False, "Invalid grade"
    try:
        m = int(rec.get('marks',-1))
        if m<0 or m>100: return False, "Invalid marks"
    except:
        return False, "Invalid marks"
    return True, ""

def require_token():
    token = request.headers.get("X-API-KEY","")
    return token==ADMIN_TOKEN

@app.route("/api/students", methods=["GET","POST"])
def api_students():
    if request.method=="GET":
        students = read_students()
        # basic filters: q (search name or id), sort, page, per_page
        q = request.args.get("q","").strip().lower()
        sort_by = request.args.get("sort","marks")
        page = int(request.args.get("page","1"))
        per_page = int(request.args.get("per_page","100"))
        filtered = students
        if q:
            filtered = [s for s in students if q in s['name'].lower() or s['id']==q]
        if sort_by in ("marks","id","name","age","grade"):
            reverse = True if sort_by=="marks" else False
            filtered = sorted(filtered, key=lambda x: x[sort_by], reverse=reverse)
        start = (page-1)*per_page
        return jsonify({"total": len(filtered), "page": page, "data": filtered[start:start+per_page]})
    else:
        if not require_token(): return jsonify({"error":"unauthorized"}), 401
        payload = request.json or {}
        ok,msg = validate_record(payload, check_unique=True)
        if not ok: return jsonify({"error":msg}), 400
        students = read_students()
        students.append({'id':payload['id'],'name':payload['name'],'age':int(payload['age']),'grade':payload['grade'].upper(),'marks':int(payload['marks'])})
        atomic_write(students)
        log_audit("create", payload['id'])
        return jsonify({"status":"ok"}), 201

@app.route("/api/students/<sid>", methods=["GET","PUT","DELETE"])
def api_student(sid):
    students = read_students()
    idx = next((i for i,s in enumerate(students) if s['id']==sid), None)
    if request.method=="GET":
        if idx is None: return jsonify({"error":"not found"}), 404
        return jsonify(students[idx])
    if request.method=="DELETE":
        if not require_token(): return jsonify({"error":"unauthorized"}), 401
        if idx is None: return jsonify({"error":"not found"}), 404
        removed = students.pop(idx)
        atomic_write(students)
        log_audit("delete", sid)
        return jsonify({"status":"deleted"})
    if request.method=="PUT":
        if not require_token(): return jsonify({"error":"unauthorized"}), 401
        if idx is None: return jsonify({"error":"not found"}), 404
        payload = request.json or {}
        # allow partial updates
        s = students[idx]
        s['name'] = payload.get('name', s['name'])
        if 'age' in payload: s['age'] = int(payload['age'])
        if 'grade' in payload: s['grade'] = payload['grade'].upper()
        if 'marks' in payload: s['marks'] = int(payload['marks'])
        ok,msg = validate_record(s, check_unique=False)
        if not ok: return jsonify({"error":msg}), 400
        atomic_write(students)
        log_audit("update", sid)
        return jsonify({"status":"updated"})

def compute_analysis(students):
    marks = [s['marks'] for s in students]
    if not marks:
        return {"count":0}
    avg = statistics.mean(marks)
    med = statistics.median(marks)
    std = statistics.pstdev(marks)
    highest = max(students, key=lambda x:x['marks'])
    lowest = min(students, key=lambda x:x['marks'])
    below = len([m for m in marks if m < avg])
    passed = len([m for m in marks if m>=50])
    pass_rate = passed/len(marks)*100
    # grade summary
    grade_b = {}
    for s in students:
        grade_b.setdefault(s['grade'], []).append(s['marks'])
    grade_summary = {g: {"count":len(v), "avg": statistics.mean(v)} for g,v in grade_b.items()}
    top3 = sorted(students, key=lambda x:x['marks'], reverse=True)[:3]
    bottom3 = sorted(students, key=lambda x:x['marks'])[:3]
    return {
        "count": len(students), "average": avg, "median": med, "std": std,
        "highest": highest, "lowest": lowest, "below_avg": below, "pass_rate": pass_rate,
        "grade_summary": grade_summary, "top3": top3, "bottom3": bottom3
    }

@app.route("/api/analyze", methods=["GET"])
def api_analyze():
    students = read_students()
    summary = compute_analysis(students)
    return jsonify(summary)

@app.route("/ui/analyze")
def ui_analyze():
    students = read_students()
    summary = compute_analysis(students)
    report = []
    if summary.get("count",0)==0:
        report = "No data available."
    else:
        report.append(f"Records: {summary['count']}")
        report.append(f"Average: {summary['average']:.2f}")
        report.append(f"Median: {summary['median']}")
        report.append(f"Std Dev: {summary['std']:.2f}")
        report.append(f"Top: {summary['highest']['name']} ({summary['highest']['marks']})")
        report.append(f"Bottom: {summary['lowest']['name']} ({summary['lowest']['marks']})")
        report.append(f"Pass Rate: {summary['pass_rate']:.2f}%")
    return render_template_string(INDEX_HTML, body=render_template_string(ANALYZE_HTML, report="\n".join(report)))

@app.route("/chart/marks.png")
def chart_marks():
    students = read_students()
    marks = [s['marks'] for s in students]
    if not marks:
        buf = io.BytesIO(); plt.figure(figsize=(6,3)); plt.text(0.5,0.5,"No data",ha='center'); plt.axis('off'); plt.savefig(buf, format='png'); buf.seek(0)
        return send_file(buf, mimetype='image/png')
    plt.figure(figsize=(6,3))
    plt.hist(marks, bins=10)
    plt.title("Marks Distribution")
    plt.xlabel("Marks"); plt.ylabel("Count")
    buf = io.BytesIO(); plt.tight_layout(); plt.savefig(buf, format='png'); buf.seek(0); plt.close()
    return send_file(buf, mimetype='image/png')

@app.route("/chart/grades.png")
def chart_grades():
    students = read_students()
    grades = {}
    for s in students:
        grades[s['grade']] = grades.get(s['grade'],0)+1
    labels = list(sorted(grades.keys()))
    values = [grades[k] for k in labels]
    plt.figure(figsize=(6,3))
    plt.bar(labels, values)
    plt.title("Grade Distribution")
    buf = io.BytesIO(); plt.tight_layout(); plt.savefig(buf, format='png'); buf.seek(0); plt.close()
    return send_file(buf, mimetype='image/png')

@app.route("/ui")
def ui_index():
    return render_template_string(INDEX_HTML, body="<p>Welcome — use the menu links.</p>")

@app.route("/ui/view")
def ui_view():
    students = read_students()
    return render_template_string(INDEX_HTML, body=render_template_string(VIEW_HTML, students=students))

@app.route("/ui/add", methods=["GET","POST"])
def ui_add():
    if request.method=="GET":
        return render_template_string(INDEX_HTML, body=ADD_HTML)
    # POST
    form = request.form
    rec = {'id':form.get('id','').strip(), 'name':form.get('name','').strip(),
           'age':form.get('age','').strip(), 'grade':form.get('grade','').strip().upper(),
           'marks':form.get('marks','').strip()}
    ok,msg = validate_record(rec, check_unique=True) if 'validate_record' in globals() else (True,"")
    # reuse validate_record if present, else basic checks
    if not ok:
        return render_template_string(INDEX_HTML, body=f"<p>Error: {msg}</p>"+ADD_HTML)
    students = read_students()
    students.append({'id':rec['id'],'name':rec['name'],'age':int(rec['age']),'grade':rec['grade'],'marks':int(rec['marks'])})
    atomic_write(students)
    log_audit("create_ui", rec['id'])
    return redirect(url_for('ui_view'))

@app.route("/ui/edit/<sid>", methods=["GET","POST"])
def ui_edit(sid):
    students = read_students()
    idx = next((i for i,s in enumerate(students) if s['id']==sid), None)
    if idx is None:
        return render_template_string(INDEX_HTML, body="<p>Student not found.</p>")
    s = students[idx]
    if request.method=="GET":
        return render_template_string(INDEX_HTML, body=render_template_string(EDIT_HTML, s=s))
    form = request.form
    if form.get('name'): s['name']=form.get('name').strip()
    if form.get('age'): s['age']=int(form.get('age'))
    if form.get('grade'): s['grade']=form.get('grade').strip().upper()
    if form.get('marks'): s['marks']=int(form.get('marks'))
    atomic_write(students)
    log_audit("update_ui", sid)
    return redirect(url_for('ui_view'))

@app.route("/ui/delete/<sid>", methods=["GET","POST"])
def ui_delete(sid):
    students = read_students()
    idx = next((i for i,s in enumerate(students) if s['id']==sid), None)
    if idx is None:
        return render_template_string(INDEX_HTML, body="<p>Student not found.</p>")
    if request.method=="GET":
        return render_template_string(INDEX_HTML, body=f"<p>Delete {students[idx]['name']}? <form method='post'><input type='submit' value='Delete'></form></p>")
    students.pop(idx)
    atomic_write(students)
    log_audit("delete_ui", sid)
    return redirect(url_for('ui_view'))

@app.route("/ui/export")
def ui_export():
    students = read_students()
    buf = io.StringIO()
    writer = csv.writer(buf)
    for s in students:
        writer.writerow([s['id'], s['name'], s['age'], s['grade'], s['marks']])
    buf.seek(0)
    return send_file(io.BytesIO(buf.getvalue().encode('utf-8')), mimetype='text/csv', as_attachment=True, download_name='students_export.csv')

@app.route("/ui/seed")
def ui_seed():
    # quick demo seed
    sample = [
        ('101','Ali Ahmed',17,'A',92),
        ('102','Sana Khan',18,'B',76),
        ('103','Zara Ali',16,'A',95),
        ('104','Hassan R',17,'C',58),
        ('105','Ayesha M',18,'B',69),
        ('106','Bilal',19,'D',44),
        ('107','Nadia',17,'A',88),
    ]
    atomic_write([{'id':a,'name':b,'age':c,'grade':d,'marks':e} for a,b,c,d,e in sample])
    log_audit("seed","demo")
    return redirect(url_for('ui_view'))

@app.route("/api/backup", methods=["POST"])
def api_backup():
    if not require_token(): return jsonify({"error":"unauthorized"}), 401
    target = backup_now()
    return jsonify({"backup": target})

@app.route("/api/backups", methods=["GET"])
def api_list_backups():
    files = sorted(os.listdir(BACKUP_DIR)) if os.path.exists(BACKUP_DIR) else []
    return jsonify(files)

@app.route("/api/restore/<fname>", methods=["POST"])
def api_restore(fname):
    if not require_token(): return jsonify({"error":"unauthorized"}), 401
    path = os.path.join(BACKUP_DIR, fname)
    if not os.path.exists(path): return jsonify({"error":"missing"}), 404
    with LOCK:
        with open(path, "rb") as src, open(FILE_NAME, "wb") as dst:
            dst.write(src.read())
    log_audit("restore", fname)
    return jsonify({"restored": fname})

@app.route("/api/report", methods=["GET"])
def api_report():
    students = read_students()
    summary = compute_analysis(students)
    # write human readable
    lines = ["SMART STUDENT RECORD ANALYSIS REPORT", f"Generated: {datetime.utcnow().isoformat()}", ""]
    if summary.get('count',0)==0:
        lines.append("No records.")
    else:
        lines.append(f"Records: {summary['count']}")
        lines.append(f"Average: {summary['average']:.2f}")
        lines.append(f"Median: {summary['median']}")
        lines.append(f"Std Dev: {summary['std']:.2f}")
        lines.append(f"Top: {summary['highest']['name']} ({summary['highest']['marks']})")
        lines.append(f"Bottom: {summary['lowest']['name']} ({summary['lowest']['marks']})")
        lines.append(f"Pass Rate: {summary['pass_rate']:.2f}%")
    with open(REPORT_FILE, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    log_audit("report","export")
    return send_file(REPORT_FILE, as_attachment=True)

@app.route("/")
def index():
    return render_template_string(INDEX_HTML, body="<p>Use menu.</p>")

if __name__ == "__main__":
    ensure_dirs()
    app.run(host="127.0.0.1", port=5000, debug=False)
